In [1]:
import selenium
from selenium import webdriver as wb
import pandas as pd
import time

In [2]:
webD = wb.Chrome('chromedriver.exe')
webD.get('https://timesofindia.indiatimes.com/archive/year-2015,month-9.cms')

In [3]:
date_urls = [] #list for storing all date urls
#Let us extract all the urls from all the rows in the webpage calender
rows = webD.find_element_by_id('calender').find_elements_by_tag_name('td')
for row in rows:
    #Not all calender row elements contain date or url so we will extract only the url of 30 days
    if len(row.find_elements_by_tag_name('a'))==0:
        continue
    elif row.find_element_by_tag_name('a').text=='':
        continue
    else:
        date_url = row.find_element_by_tag_name('a').get_property('href')
    date_urls.append(date_url)

In [ ]:
len(date_urls)

In [5]:
urls = [] #list for storing all article urls
#Let's iterate through all the date url and extract all article url from all 30 calender days
for date_url in date_urls:
    webD.get(date_url)
    #All articls in the webpage are contained in different panels, some panels don't have any articles
    panel = webD.find_elements_by_tag_name('table')[2].find_elements_by_tag_name('td')
    for pane in panel:
        if len(pane.find_elements_by_tag_name('a'))==0:
            continue
        else:
            art_elements = pane.find_elements_by_tag_name('a')
            for art_element in art_elements:
                url = art_element.get_property('href')
                urls.append(url)

In [6]:
len(urls)

15000

In [61]:
authors = []
dates = []
verticals = []
headings = []
descs = []
for url in urls[:25]:
    webD.get(url)
    time.sleep(2)
    #Code to tackle error 404 in page
    if len(webD.find_elements_by_class_name('error'))>0:
        if webD.find_element_by_class_name('error').text=='404\nPAGE NOT FOUND':
            continue
    #Code to tackle photo stories which are not in news format
    elif len(webD.find_elements_by_class_name('photostory_heading'))>0:
        continue
    elif len(webD.find_elements_by_class_name('slide-info'))>0:
        continue
    #Code if the article is in news format
    if len(webD.find_elements_by_class_name('_3R_Dd'))>0:
        if len(webD.find_element_by_class_name('_3R_Dd').find_elements_by_tag_name('a'))>0:
            author = webD.find_element_by_class_name('_3R_Dd').find_element_by_tag_name('a').text
        else:
            author = ''
        date = webD.find_element_by_class_name('_3R_Dd').find_element_by_tag_name('div').text.split('|')[-1].strip().replace('Updated: ','').replace('Created: ','')
        
        if len(webD.find_element_by_xpath('//div[@class="_2tDZb contentwrapper "]').find_elements_by_tag_name('a'))>1:
            vertical = webD.find_element_by_xpath('//div[@class="_2tDZb contentwrapper "]').find_elements_by_tag_name('a')[1].text
        else:
            vertical = ''
        
        heading = webD.find_element_by_class_name('_2NFXP ').find_element_by_tag_name('h1').text
        desc = webD.find_element_by_class_name('ga-headlines').text
        
    #Code if the article is in other format
    if len(webD.find_elements_by_class_name('as_byline'))>0:
        #Incase author name is not mentioned
        if len(webD.find_element_by_class_name('as_byline').find_elements_by_tag_name('a'))>0:
            author = webD.find_element_by_class_name('as_byline').find_element_by_tag_name('a').text
        else:
            author = ''
        date = webD.find_element_by_class_name('as_byline').find_elements_by_tag_name('div')[-1].text.replace('Updated: ','').replace('Created: ','')
        vertical = webD.find_element_by_class_name('navbdcrumb').find_elements_by_tag_name('li')[1].text
        heading = webD.find_element_by_class_name('heading1').text
        desc = webD.find_element_by_tag_name('arttextxml').text
    authors.append(author)
    dates.append(date)
    verticals.append(vertical)
    headings.append(heading)
    descs.append(desc)
webD.quit()

In [14]:
news = pd.DataFrame({'Date':dates,'Author':authors,'Vertical':verticals,'Heading':headings,'Description':descs})

In [ ]:
news.info()

In [15]:
news

,Date,Author,Vertical,Heading,Description


In [38]:
news.to_csv('TOI_sept_2015_news.csv')